<a href="https://colab.research.google.com/github/jorgel-mendes/Portifolio_Recommender_Codenation/blob/master/notebooks/Recommender_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final challenge codenation
Last challenge for the codenation acceleration and a demonstration of my knowledge with an end to end data science project. 

Here the recommendation model is demonstrated in two ways. First it is run directly on the notebook, with the function called from github and the full market dataset.

And then the model is also called from IBM cloud function through a HTTP post request. A reduced market dataset is used for this example.

## Full market implementation

In [2]:
!wget https://raw.githubusercontent.com/jorgel-mendes/Portifolio_Recommender_Codenation/master/scr/model.py

import pandas as pd
import numpy as np
from model import get_recommendations

--2020-10-29 00:46:41--  https://raw.githubusercontent.com/jorgel-mendes/Portifolio_Recommender_Codenation/master/scr/model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3889 (3.8K) [text/plain]
Saving to: ‘model.py’

model.py            100%[===================>]   3.80K  --.-KB/s    in 0s      

2020-10-29 00:46:41 (56.1 MB/s) - ‘model.py’ saved [3889/3889]



In [3]:
from requests import get
from zipfile import ZipFile
from io import BytesIO

remotezip = get("https://codenation-challenges.s3-us-west-1.amazonaws.com/ml-leads/estaticos_market.csv.zip")
root = ZipFile(BytesIO(remotezip.content))
market_initial =  pd.read_csv(root.open('estaticos_market.csv')) 

portifolio2 = pd.read_csv("https://raw.githubusercontent.com/jorgel-mendes/Portifolio_Recommender_Codenation/dev/data/estaticos_portfolio2.csv")

portifolio_test = portifolio2[0:100]

In [ ]:
recommended_values = get_recommendations(portifolio_test, 100, market_initial)

In [ ]:
recommended_values.head()

,Unnamed: 0,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,fl_optante_simples,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,vl_potenc_cons_oleo_gas,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,...,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_ate_18,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,grau_instrucao_macro_desconhecido,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
281417,281417,566ff012a2dace92facb594a0596facbadec7e6daf9983...,True,MUNICIPIO,RN,OUTROS,"ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL",SERVIÇO,43.876712,> 20,False,False,False,False,False,2005-05-07,False,False,False,NAO,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,False,False,True,NaN,NaN,6.0,2.0,False,2.0,2.0,6.0,NaN,NaN,NaN,False,RN,AZUL,ACIMA DE 1 ANO,...,73925460.0,73925460.0,436.0,452.0,40.49,59.51,NaN,NaN,15.0,54.0,72.0,88.0,83.0,63.0,44.0,33.0,NaN,42.0,213.0,197.0,NaN,452.0,25.233333,0.0,0.0,437.0,889.0,154.560736,5014.966667,-2.066667,173.989086,514.266667,29.300000,452.0,452.0,645.0,0.000000,-29.922481,0.000000,0
112687,112687,baa56942730983a6bfa56440fbc9de7be83a50ec1e6c15...,True,EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITAD...,AC,OUTROS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,13.841096,10 a 15,False,False,False,False,False,2004-12-28,False,False,True,NAO,CONSTRUCAO DE EDIFICIOS,CONSTRUCAO,False,False,False,NaN,NaN,NaN,NaN,False,NaN,0.0,0.0,NaN,NaN,NaN,False,AC,VERDE,ACIMA DE 1 ANO,...,13136580.0,13136580.0,82.0,87.0,97.70,2.30,NaN,8.0,14.0,19.0,15.0,8.0,11.0,1.0,7.0,4.0,NaN,NaN,87.0,NaN,NaN,87.0,11.100000,15.0,1.0,83.0,173.0,13.375530,48.166667,-0.700000,19.588148,48.166667,11.100000,90.0,76.0,22.0,18.421053,309.090909,1.204819,0
297247,297247,76a3f1781e4cb1dd4fa6b931c3be08c02f70ebc730f312...,True,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,10.326027,10 a 15,False,False,False,False,False,2008-07-03,False,True,True,SIM,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,False,False,True,NaN,NaN,6.0,4.0,False,1.0,4.0,6.0,NaN,NaN,NaN,False,AM,VERDE,ACIMA DE 1 ANO,...,55946376.0,56156376.0,404.0,166.0,80.72,19.28,7.0,19.0,34.0,39.0,23.0,13.0,17.0,9.0,3.0,2.0,NaN,7.0,145.0,14.0,NaN,166.0,2.033333,48.0,37.0,406.0,572.0,19.349068,117.566667,-0.700000,35.597791,117.566667,2.033333,166.0,154.0,143.0,7.792208,16.083916,23.052960,2
366551,366551,127189bfa020a9a0aca06af21117acee792800f0063913...,True,SOCIEDADE EMPRESARIA LIMITADA,RO,ENTIDADES EMPRESARIAIS,PRODUTOS DE AGROPECUARIA,AGROPECUARIA,34.101370,> 20,False,False,False,False,False,2005-11-03,False,False,False,NAO,PRODUCAO FLORESTAL,AGRICULTURA PECUARIA PRODUCAO FLORESTAL PESCA ...,False,True,True,NaN,NaN,2.0,5.0,True,NaN,5.0,2.0,NaN,NaN,5.0,False,RO,VERDE,ACIMA DE 1 ANO,...,3039444.0,3039444.0,100.0,20.0,80.00,20.00,NaN,NaN,NaN,2.0,4.0,3.0,3.0,5.0,1.0,2.0,NaN,13.0,5.0,2.0,NaN,20.0,4.033333,2.0,1.0,100.0,120.0,17

In [ ]:
values_in_portifolio = np.intersect1d(portifolio2['id'], recommended_values['id']).shape[0]
print(values_in_portifolio, "values were in the portifolio.")

22 values were in the portifolio.


## Demo implementation

In [ ]:
try:
  portifolio2
except:
  portfolio2 = pd.read_csv("https://raw.githubusercontent.com/jorgel-mendes/Portifolio_Recommender_Codenation/dev/data/estaticos_portfolio2.csv")

portifolio_demo = {'id': portifolio2.loc[:100, 'id'].to_dict(),
 'n_recommendations': 50}

In [ ]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

data = json.dumps(portifolio_demo)

response = requests.post('https://us-south.functions.appdomain.cloud/api/v1/web/jorgel.mendes%40hotmail.com_dev/default/portifolio_recommender.json', headers=headers, data=data)

id_recommended_companies = pd.Series(response.json()['id'], name = 'id')

In [ ]:
market_demo = pd.read_csv('https://raw.githubusercontent.com/jorgel-mendes/Portifolio_Recommender_Codenation/dev/data/market_demo.csv')
recommended_companies = market_demo[market_demo['id'].isin(id_recommended_companies)]
recommended_companies.head()

,Unnamed: 0,Unnamed: 0.1,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,fl_optante_simples,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,vl_potenc_cons_oleo_gas,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,...,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_ate_18,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,grau_instrucao_macro_desconhecido,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
101,67692,67692,ca7d4d28883a04fa6ffda83f022de343847cb45bc9879f...,True,MUNICIPIO,PI,OUTROS,"ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL",SERVIÇO,21.835616,> 20,False,False,False,False,False,2005-11-03,False,False,False,NAO,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,False,False,True,NaN,NaN,12.0,7.0,False,2.0,7.0,12.0,NaN,NaN,NaN,False,PI,AZUL,...,30291408.0,30291408.0,42.0,214.0,41.12,58.88,NaN,3.0,14.0,40.0,52.0,36.0,25.0,18.0,12.0,14.0,NaN,7.0,114.0,93.0,NaN,214.0,28.300000,0.0,0.0,42.0,256.0,111.728125,423.966667,0.033333,119.898287,423.966667,28.300000,214.0,214.0,214.0,0.000000,0.000000,0.000000,0
152,108941,108941,425f2716986de6a09a8a114f1e8b9d14595e6f31b70542...,True,MUNICIPIO,PI,OUTROS,"ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL",SERVIÇO,25.884932,> 20,False,False,False,False,False,2004-06-12,False,True,True,NAO,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,False,False,True,NaN,NaN,9.0,8.0,False,3.0,8.0,9.0,NaN,NaN,NaN,False,PI,VERDE,...,38224870.0,38224870.0,52.0,269.0,44.61,55.39,NaN,3.0,14.0,40.0,41.0,65.0,39.0,32.0,18.0,17.0,NaN,96.0,65.0,108.0,NaN,269.0,28.300000,0.0,0.0,52.0,321.0,144.040498,314.333333,0.400000,161.442627,314.333333,28.300000,269.0,269.0,269.0,0.000000,0.000000,0.000000,0
153,109311,109311,5d612f9b7ce15926a6f86a7154211f812a91ab239efa54...,True,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,SERVICOS ADMINISTRATIVOS,SERVIÇO,24.824658,> 20,False,False,False,False,False,2005-11-03,False,False,False,SIM,SERVICOS PARA EDIFICIOS E ATIVIDADES PAISAGIST...,ATIVIDADES ADMINISTRATIVAS E SERVICOS COMPLEME...,False,False,True,NaN,NaN,26.0,1.0,False,NaN,1.0,26.0,NaN,NaN,3.0,False,AM,AZUL,...,30858084.0,30858084.0,5805.0,282.0,55.67,44.33,NaN,2.0,29.0,57.0,55.0,57.0,29.0,22.0,20.0,11.0,NaN,204.0,73.0,5.0,NaN,282.0,24.200000,0.0,0.0,5808.0,6091.0,31.202415,240.500000,-0.466667,87.386219,236.300000,24.200000,283.0,283.0,1352.0,0.000000,-79.068047,0.000000,0
154,109461,109461,fb70b694ed5ce5337ff9fe8f8bfb8ff565b6b3026d4768...,True,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...,AC,ADMINISTRACAO PUBLICA,"ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL",SERVIÇO,43.800000,> 20,False,False,False,False,False,1998-06-23,False,True,True,NAO,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,False,False,True,NaN,NaN,16.0,2.0,False,7.0,2.0,16.0,NaN,NaN,NaN,False,AC,VERDE,...,327178100.0,327178100.0,403.0,638.0